Увеличение количества выборки

In [1]:
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as nafc
from nlpaug.util import Action

In [31]:
from tqdm import tqdm_notebook

In [23]:
intents = {
    'Когда я получу полис дмс' : 'Полис будет доступен с первого числа следующего месяца по окончанию испытательного срока.',
    'Я могу застраховать родственников' : 'Застраховать можно не более трех родственников.',
    'Когда я могу застраховать родственников по дмс' : 'В течение двух месяцев от даты получения Вашего полиса.',
    'Как застраховать новобрачного мужа или жену' : 'Заявление оформляется в течение двух месяцев с даты регистрации брака.',
    'Как застраховать новорожденного ребенка' : 'Заявление оформляется в течение двух месяцев с даты рождения ребенка',
    'От каких рисков я застрахован по дмс' : 'Амбулаторно-поликлиническое обслуживание, в том числе дистанционные онлайн - консультации, выезд врача на дом, стоматологическая помощь, скорая медицинская помощь, госпитализация.',
    'Как работает дмс в другом городе' : 'дмс в других городах работает только в экстренных случаях. Предусмотрено оказание экстренной и неотложной помощи на территории Российской Федерации.',
    'Где я могу записаться на пробную тренировку' : 'Вы можете записаться на портале СОГАЗа или по номеру два, один, ноль, ноль.',
    'Как оформить абонемент в спорт зал' : 'Вам необходимо заполнить заявление на портале Galileo. После передать скан заявления ответственному сотруднику.',
    'Сколько стоит абонемент в спорт зал' : 'Стоимость абонемента – одиннадцать тысяч пятьсот рублей за год. При оформлении на неполный период стоимость уменьшается',
    'Когда начнёт действовать абонемент в спорт зал' : 'Aбонемент начнёт действовать через два рабочих дня после его оформления.',
    'Компенсируют расходы на детский лагерь' : 'Компенсация детского отдыха может предоставляться при соблюдении условий и порядка оформления',
    'Я могу получить компенсацию детского лагеря': 'Компенсация детского отдыха может предоставляться при соблюдении условий и порядка оформления',
    'Как работает компенсация детского лагеря' : 'Сотруднику необходимо выбрать лагерь самостоятельно и оплатить путевку. После возвращения ребенка, нужно предоставить комплект документов для получения компенсации',
    'Какая компенсация детского лагеря' : 'Компенсация составляет пятьдесят процентов от стоимости путевки, но не более двадцати одной тысячи ста пятидесяти рублей за одну путевку',
    'Компенсировать можно только одну путёвку' : 'Получить компенсацию можно за две путевки при условии наличия двух и более детей',
    'За какого положена компенсация' : 'Компенсация положена только за детей сотрудника от семи до шестнадцати лет.',
    'Когда можно компенсировать путевку' : 'Компенсировать детский лагерь можно один раз в два календарных года',
    'Какие детские лагеря можно компенсировать' : 'Только которые расположены на территории РФ.',
    'Что нужно для компенсации детского лагеря' : 'Необходимо заявление, договор с лагерем, Чек или квитанция, Лицензия лагеря при наличии, Свидетельство о рождении.',
    'Как получить компенсацию детского лагеря' : 'Заявление и документы на получение компенсации подаются не позднее двух месяцев с даты возвращения ребенка из детского лагеря',
    'Когда я получу полис дмс' : 'Сотрудники, успешно прошедшие испытательный срок, включаются в корпоративный договор дмс с первого числа месяца, следующего за датой окончания испытательного срока',
    'Сколько всего родственников я могу застраховать в дмс' : 'На условиях корпоративного договора сотрудник имеет право застраховать не более трех родственников',
    'Я могу застраховать родственников в корпоративный договор дмс в любой момент' : 'На оформление программ страхования для близких родственников у сотрудника есть два месяца с даты начала действия собственного полиса',
    'Какие риски входят в мою программу Страхования жизни' : 'В рамках корпоративной программы Страхования жизни сотрудники застрахованы по рискам «Смерть» и «Инвалидность».',
    'Если я поеду в другой город смогу ли я воспользоваться полисом дмс' : 'Полисом ДМС предусмотрено оказание экстренной и неотложной помощи на территории Российской Федерации',
    'где могу записаться на пробную тренировку' : 'Вы можете записаться на портале СОГАЗа или по номеру два, один, ноль, ноль.',
    'Когда начнёт действовать абонемент в спорт зал' : 'Программа начнет действовать через два рабочих дня после подачи заявления на подключение к программе',
    'Как работает компенсация детского лагеря' : 'Сотрудник сам выбирает лагерь, в котором будет отдыхать ребенок, заключает договор с лагерем и самостоятельно оплачивает стоимость путевки. После возвращения ребенка из лагеря сотрудник предоставляет комплект документов для получения компенсации.',
    'Какие документы необходимо предоставить для компенсации детского лагеря' : 'Заявление, Договор с лагерем, Документы подтверждающие оплату путевки сотрудником, Свидетельство о рождении ребенка.'
}

In [33]:
augmented_text = []
answers = []
for intent in tqdm_notebook(intents):
    aug = naw.SynonymAug(aug_src='ppdb', model_path='ppdb-1.0-xxl-lexical', lang='ru', aug_p=0.1)
    augmented_text += aug.augment(intent, n=10000)
    answers += [intents[intent] for _ in range(10000)]
    aug = naw.SynonymAug(aug_src='ppdb', model_path='ppdb-1.0-xxl-lexical', lang='ru', aug_p=0.2)
    augmented_text += aug.augment(intent, n=10000)
    answers += [intents[intent] for _ in range(10000)]

C:\Users\User\AppData\Local\Temp\ipykernel_11884\3620093649.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for intent in tqdm_notebook(intents):


  0%|          | 0/27 [00:00<?, ?it/s]